In [1]:
import time, gym, sys, torch
from torch import nn
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
from scipy.stats import geom
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import homework.hw1.load_policy as load_policy
import homework.hw1.tf_util as tf_util
import tensorflow as tf
from copy import deepcopy as dc

sigma = 0.1

dim = 17
dim_act = 6
obs_min = np.array([-0.25,-0.8 ,-0.7 ,-0.7,-0.65,-0.75,-0.95,-0.6 ,-1.5,-3.1,-7.1,-20,-24,-27,-27,-30,-20])
obs_max = np.array([ 0.4 , 1.65, 0.95, 0.9, 0.95, 0.95, 1.1 , 0.75, 8  , 3.4, 7  , 19, 25, 22, 25, 32, 26])

/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/

In [2]:
"""

initialise NN

"""

# Hyperparameters for our network
input_size = dim
n_layer = 3
hidden_sizes = [10,10,10]
output_size = 1

# # Store hyperparameters in string
# hyps = str(hidden_sizes[0])
# for l in range(1,n_layer):
#     hyps = hyps+'_'+str(hidden_sizes[l])

# # Build a feed-forward network
# model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
#                       nn.Tanh(),
#                       nn.Linear(hidden_sizes[0], hidden_sizes[1]),
#                       nn.Tanh(),
#                       nn.Linear(hidden_sizes[1], hidden_sizes[2]),
#                       nn.Tanh(),
#                       nn.Linear(hidden_sizes[-1], output_size))


# # Get weights
# weights = []
# biases = []
    
# for l in range(n_layer+1):
#     weights.append(np.zeros(model[2*l].weight.shape))
#     biases.append(np.zeros(model[2*l].bias.shape))

# # Set all weights to 0
# for l in range(n_layer+1):
#     model[2*l].weight = torch.nn.Parameter(torch.from_numpy(0*weights[l]).float(), requires_grad=True)
#     model[2*l].bias = torch.nn.Parameter(torch.from_numpy(0*biases[l]).float(), requires_grad=True)

In [3]:
"""

INITIALISE ACTION AND OBSERVATION SPACE

"""

actions = []
actions_matrix = np.load('HC_actions.npy')
N_a = 8
for i in range(N_a):
    actions.append(actions_matrix[i,:])

In [4]:
"""

FUNCTIONS - PRIOR

""" 

    
''' Set sigma_w and sigma_b uniformly '''    
alpha = 1.5
sigma_w_sq = np.ones(n_layer+1)*2
sigma_b_sq = np.ones(n_layer+1)*2
    
    
# ''' define scaling/covariance matrix '''
# C = [[],[]]
# C_root = [[],[]]
# C_arr = np.ones(model[0].weight.shape)
# for t in range(hidden_sizes[0]):
#     for s in range(input_size):
#         C_arr[t][s] = sigma_w_sq[0]/np.power(t+1,alpha)
# C[0].append(C_arr)
# C_root[0].append(C_arr**(1/2))

# for l in range(1,n_layer):
#     C_arr = np.ones(model[2*l].weight.shape)
#     for t in range(hidden_sizes[l]):
#         for s in range(hidden_sizes[l-1]):
#             C_arr[t][s] = sigma_w_sq[l]/np.power(s+1,alpha)/np.power(t+1,alpha)
#     C[0].append(C_arr)
#     C_root[0].append(C_arr**(1/2))
    
# C_arr = np.ones(model[2*n_layer].weight.shape)   
# for t in range(output_size):
#     for s in range(hidden_sizes[n_layer-1]):
#         C_arr[t][s] = sigma_w_sq[n_layer]/np.power(s+1,alpha)
# C[0].append(C_arr)   
# C_root[0].append(C_arr**(1/2))

# C_arr = np.ones(model[0].bias.shape)
# for t in range(hidden_sizes[0]):
#     C_arr[t] = sigma_b_sq[0]/np.power(t+1,alpha)
# C[1].append(C_arr)
# C_root[1].append(C_arr**(1/2))

# for l in range(1,n_layer):
#     C_arr = np.ones(model[2*l].bias.shape)
#     for t in range(hidden_sizes[l]):
#         C_arr[t] = sigma_b_sq[l]/np.power(t+1,alpha)
#     C[1].append(C_arr)
#     C_root[1].append(C_arr**(1/2))

# C_arr = np.ones(model[2*n_layer].bias.shape)
# for t in range(output_size):
#     C_arr[t] = sigma_b_sq[n_layer]/np.power(t+1,alpha)
# C[1].append(C_arr)
# C_root[1].append(C_arr**(1/2))


''' prior samples are represented by array of coefficients '''
def sample_prior(c=1):
    w = []
    b = []
    
    ''' weights '''
    w_layer = np.random.randn(model[0].weight.shape[0],model[0].weight.shape[1])*C_root[0][0]
    w.append(c*w_layer)
    
    for l in range(1,n_layer):
        w_layer = np.random.randn(model[2*l].weight.shape[0],model[2*l].weight.shape[1])*C_root[0][l]
        w.append(c*w_layer)
     
    w_layer = np.random.randn(model[2*n_layer].weight.shape[0],model[2*n_layer].weight.shape[1])*C_root[0][-1]
    w.append(c*w_layer)
                
    ''' biases '''
    b_layer = np.random.randn(model[0].bias.shape[0])*C_root[1][0]
    b.append(c*b_layer)
    
    for l in range(1,n_layer):
        b_layer = np.random.randn(model[2*l].bias.shape[0])*C_root[1][l]
        b.append(c*b_layer)
     
    b_layer = np.random.randn(model[2*n_layer].bias.shape[0])*C_root[1][-1]
    b.append(c*b_layer)
    
    return [w,b]


''' evaluate the log_prior up to a constant '''
def logprior(xi):
    log_prior = 0
    w = xi[0]
    b = xi[1]
    
    ''' weights '''
    for l in range(n_layer+1):
        log_prior += -0.5*np.sum(w[l]**2/C[0][l])
        
    ''' biases '''
    for l in range(n_layer+1):
        log_prior += -0.5*np.sum(b[l]**2/C[1][l])
        
    return log_prior


def prior_proposal(xi):
    log_prior_xi = logprior(xi)
    
    w = xi[0]
    b = xi[1]
    
    ''' Firstly make a cpoy of the current state '''
    w_proposal = []
    b_proposal = []
    for l in range(n_layer+1):
        w_proposal.append(dc(w[l]))
        b_proposal.append(dc(b[l]))
    
    random_l = np.random.randint(n_layer)
    
    ''' Sample index of function to be swapped '''
    random_i = geom.rvs(1/alpha)
    while random_i>=model[2*random_l].weight.shape[0]-1:  
        random_i = geom.rvs(1/alpha)
        
    ''' swap f^l_i with f^l_{i+1}'''
    w_proposal[random_l][random_i,:] = w[random_l][random_i+1,:]
    w_proposal[random_l][random_i+1,:] = w[random_l][random_i,:]
    w_proposal[random_l+1][:,random_i] = w[random_l+1][:,random_i+1]
    w_proposal[random_l+1][:,random_i+1] = w[random_l+1][:,random_i]
    b_proposal[random_l][random_i] = b[random_l][random_i+1]
    b_proposal[random_l][random_i+1] = b[random_l][random_i]
    
    xi_proposal = [w_proposal,b_proposal]
    log_prior_proposal = logprior(xi_proposal)
    
    a = np.exp(log_prior_proposal-log_prior_xi)
    if np.random.uniform() < a:
        counts[0] += 1
        return xi_proposal
    else:
        counts[1] += 1
        return xi
    
counts = np.zeros(2)

In [5]:
"""

FUNCTIONS - LIKELIHOOD 

"""

''' Function to integrate within likelihood '''
def f(x,v_a):
    value = norm._pdf((x-v_a[0])/sigma)/sigma
    for i in range(1,N_a):
        value = value*norm._cdf((x-v_a[i])/sigma)
    return value


def likelihood(pair):
    poss = pair[0:8]
    vels = pair[8:17]
    action = int(pair[17])
    poss = np.insert(poss,0,0)
    
    ''' Check where all actions would take us '''
    x = np.zeros((N_a,dim))
    v = np.zeros(N_a)
    for j in range(N_a):
        env.set_state(poss, vels)
        x[j,:] = env.step(actions[j])[0]
        v[j] = u(x[j,:]).detach().numpy()[0]
        
    ''' Sort v such that the first entry is the taken action '''
    v_a = np.zeros(N_a)
    if action!=0:
        v_a[0] = v[action]
        v_a[1:action+1] = v[0:action]
        v_a[action+1:] = v[action+1:]
    else:
        v_a = v
            
    ''' Integrate over pdf of chosen action and cdf of other actions '''
    lklhd = quad(f,v_a[0]-3*sigma,v_a[0]+3*sigma,args=v_a,limit=500)[0]
    return lklhd

def loglikelihood(data):
    loglikelihood = 0
    for j in range(data.shape[0]):
        loglikelihood += np.log(likelihood(data[j,:]))
    return loglikelihood

''' function which is integrated in likelihood gradient '''
def f_grad(t,args):
    v_a = args[0]
    j = args[1]
    if j==0:
        value = (t-v_a[0])/(sigma**2)*norm._pdf((t-v_a[0])/sigma)/sigma
    else:
        value = -norm._pdf((v_a[0]-v_a[j])/(np.sqrt(2)*sigma))/(np.sqrt(2)*sigma)*norm._pdf((t-(v_a[0]+v_a[j])/2)/(sigma/np.sqrt(2)))/(sigma/np.sqrt(2))
    for i in range(1,N_a):
        if i!=j:
            value = value*norm._cdf((t-v_a[i])/sigma)
    return value

''' partial derivative dl/dv '''
def grad_ll(v_a,j):
    if j==0:
        return quad(f_grad,v_a[0]-3*sigma,v_a[0]+3*sigma,args=[v_a,j],limit=500)[0]
    else:
        return quad(f_grad,(v_a[0]+v_a[j])/2-3*sigma,(v_a[0]+v_a[j])/2+3*sigma,args=[v_a,j],limit=500)[0]

    
def diff_ll(data):
    ''' One (more efficient) gradient implementation '''
    diff_w = []
    diff_b = []
        
    x = np.zeros((N_a,dim))
    v = np.zeros(N_a)
    x_a = np.zeros((N_a,dim))
    v_a = np.zeros(N_a)
    grad_a = np.zeros(N_a)
    data_state = np.zeros(dim)
    
    ''' Iterate through all or a subset of the data points, and compute the respective gradients '''
    if stochastic_gradients and unadjusted:
        range_i = random.sample(range(data.shape[0]),10)
    else:
        range_i = range(data.shape[0])
    for i in range_i:
        lh = likelihood(data[i,:])
        data_poss = data[i,0:8]
        data_vels = data[i,8:17]
        data_poss = np.insert(data_poss,0,0)
        data_action = int(data[i,-1])
        
        for j in range(N_a):
            env.set_state(data_poss, data_vels)
            x[j,:] = env.step(j)[0]
            v[j] = u(x[j,:]).detach().numpy()[0]
        
        # sort v and x such that the first entry is the taken action
        if data_action!=0:
            v_a[0] = v[data_action]
            v_a[1:data_action+1] = v[0:data_action]
            v_a[data_action+1:] = v[data_action+1:]
            x_a[0,:] = x[data_action,:]
            x_a[1:data_action+1,:] = x[0:data_action,:]
            x_a[data_action+1:,:] = x[data_action+1:,:]
        else:
            v_a = v
            x_a = x
            
        for j in range(N_a):
            grad_a[j] = grad_ll(v_a,j)/lh
        mean_grad = np.mean(grad_a)
        grad_a -= mean_grad
            
        for j in range(N_a):
            out = u(x_a[j,:])
            out.backward(torch.from_numpy(np.array([grad_a[j]])).float())

    for l in range(n_layer+1):
        diff_w.append(model[2*l].weight.grad.data.clone().detach().numpy())
        diff_b.append(model[2*l].bias.grad.data.clone().detach().numpy())
        
    return [diff_w, diff_b]

In [6]:
"""

FUNCTIONS - VALUE FUNCTION AND POLICIES

"""

''' u(x), which evaluates the function u at x=(pos,speed) with the currently set xi '''
def u(x):
    x_transformed = np.zeros(dim)
    for i in range(dim):
        x_transformed[i] = ((x[i]-obs_min[i])/(obs_max[i]-obs_min[i])-1/2)*2
        
    value = model(torch.from_numpy(x_transformed).float())
    return value
    
''' Policy from berkeley course '''
def policy(obs):
    return policy_fn(obs[None,:])
print('loading and building expert policy')
policy_fn = load_policy.load_policy('/Users/torbensell/Dropbox (Cambridge University)/UNI/CAM/PhD/Programme/BIRL_NN_pCN/homework/hw1/experts/HalfCheetah-v2.pkl')
print('loaded and built')

loading and building expert policy
obs (1, 17) (1, 17)
loaded and built


In [7]:
"""

FUNCTIONS - ANALYTICS

"""

''' Progress bar to know how much longer one has to wait '''
def progressBar(t,value, t_max, acceptances, bar_length=40):
    percent = float(t) / t_max
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write("\rIteration: {0}    Acceptance ratio: {1}    Percent: [{2}] {3}%  ".format(value,round(acceptances/value,3),arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()      
    
''' Plotting a value function '''    
def func_plot(xi,name):
    x = np.arange(obs_min[0],obs_max[0],0.02)
    y = np.arange(obs_min[1],obs_max[1],0.05)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for l in range(n_layer+1):
        model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi[0][l]).float(), requires_grad=False)
        model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi[1][l]).float(), requires_grad=False)
        
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i,j] = u((X[i,j],Y[i,j],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0))[0]
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.RdBu,linewidth=0, antialiased=False)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    ax.set_xlabel('x-axis')
    ax.set_ylabel('y-axis')
    ax.set_zlabel('z-axis')
    ax.view_init(elev=25, azim=-120)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)
    
def trajectory_plot(xi,name):
    x = np.arange(len(xi))
    fig = plt.figure()
    plt.plot(x,xi)
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)
    
def autocorr(x,lags):
    mean=np.mean(x)
    var=np.var(x)
    xp=x-mean
    corr=[1. if l==0 else np.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]
    return np.array(corr)

''' Calculate the Effective Sample Size, assumes algorithm already burned in '''
def ESS(logposterior,name):
    fig, ax = plt.subplots()
    N = len(logposterior)
    ax.stem(autocorr(logposterior, range(int(N*0.1))),use_line_collection=True) 
    ESS = N/(1+2*sum(autocorr(logposterior, range(int(N*0.1)))))
    print('\nEffective Sample Size:', round(ESS))
    print('Samples required to generate 1 independent sample:', round(N/ESS,2))
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)    

In [8]:
"""

Uncertainty Quantification Initialisation

"""

x_test = np.load('HC_x_test.npy')
v_test = []
for i in range(N_a):
    v_test.append([])

x_test_100 = np.load('HC_x_test_100.npy')
a_test_100 = np.load('HC_a_test_100.npy')
optimal_choice = 0
not_optimal_choice = 0
v_all = np.zeros((N_a,100))

def test_value_fn():
    for i in range(N_a):
        v = u(x_test[i,:]).detach().numpy()[0]
        v_test[i].append(v)
        if i>0:
            v_test[i][-1]=v_test[i][-1]-v_test[0][-1]
    v_test[0][-1] = 0
        
    global optimal_choice
    global not_optimal_choice
    v = np.zeros(N_a)
    for j in range(100):
        for i in range(N_a):
            v[i] = u(x_test_100[i,:,j]).detach().numpy()[0]
            v_all[i,j] += v[i]
        if a_test_100[j]==np.argmax(v):
            optimal_choice += 1
        else:
            not_optimal_choice += 1

        
def boxplot_value_fn():
    global v_test
    global optimal_choice
    global not_optimal_choice
    
    fig, ax = plt.subplots()
    ax.set_title('UQ of relative value function evaluation')
    reward_test = np.load('HC_reward_test.npy')
    tags = []
    for i in range(N_a):
        tags.append(str(round(reward_test[i],2)))
    ax.set_xticklabels(tags,rotation=45)
    ax.boxplot(v_test)
    fig.savefig('figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_UQ.pdf', dpi=300)
    plt.close(fig)
    
    fig, ax = plt.subplots()
    ax.set_title('UQ of relative value function evaluation')
    ax.boxplot(v_test)
    fig.savefig('figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_UQ_v2.pdf', dpi=300)
    plt.close(fig)
    
    v_test = []
    for i in range(N_a):
        v_test.append([])
        
    print('Samples, Correct choices: ',optimal_choice/(optimal_choice+not_optimal_choice))
    print('Samples, Wrong choices:   ',not_optimal_choice/(optimal_choice+not_optimal_choice))
    
    ''' Mean values '''
    optimal_choice = 0
    not_optimal_choice = 0
    
    for j in range(100):
        for i in range(N_a): 
            if a_test_100[j]==np.argmax(v_all[:,j]):
                optimal_choice += 1
            else:
                not_optimal_choice += 1
                
    print('Mean,    Correct choices: ',optimal_choice/(optimal_choice+not_optimal_choice))
    print('Mean,    Wrong choices:   ',not_optimal_choice/(optimal_choice+not_optimal_choice))

In [9]:
"""

FUNCTIONS - MCMC (pCN/pCNL)

"""
    
def acceptance_prop(xi_u, xi_v,data,ll_u,diff_u=False):
    accept_prop = -ll_u
    ll_v = 0
    
    if method=='pCNL' or method=='CNL':
        for l in range(n_layer+1):
            model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi_v[0][l]).float(), requires_grad=True)
            model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi_v[1][l]).float(), requires_grad=True)
    else:
        for l in range(n_layer+1):
            model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi_v[0][l]).float(), requires_grad=False)
            model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi_v[1][l]).float(), requires_grad=False)
        
    for j in range(data.shape[0]):
        lh = likelihood(data[j,:])
        ll_v += np.log(lh)
    accept_prop += ll_v
    
    ''' unadjusted? '''
    if unadjusted:
        diff_v = diff_ll(data)
        return 1,ll_v,diff_v
        
    elif method=='pCNL':
        diff_v = diff_ll(data)
        for l in range(n_layer+1):
            accept_prop += -np.sum((xi_v[0][l]-xi_u[0][l])*diff_u[0][l])/2 - delta*np.sum((xi_u[0][l]+xi_v[0][l])*diff_u[0][l])/4 + delta*np.linalg.norm(diff_u[0][l]*C_root[0][l])**2/4
            accept_prop += -np.sum((xi_v[1][l]-xi_u[1][l])*diff_u[1][l])/2 - delta*np.sum((xi_u[1][l]+xi_v[1][l])*diff_u[1][l])/4 + delta*np.linalg.norm(diff_u[1][l]*C_root[1][l])**2/4

            accept_prop -= -np.sum((xi_u[0][l]-xi_v[0][l])*diff_v[0][l])/2 - delta*np.sum((xi_v[0][l]+xi_u[0][l])*diff_v[0][l])/4 + delta*np.linalg.norm(diff_v[0][l]*C_root[0][l])**2/4  
            accept_prop -= -np.sum((xi_u[1][l]-xi_v[1][l])*diff_v[1][l])/2 - delta*np.sum((xi_v[1][l]+xi_u[1][l])*diff_v[1][l])/4 + delta*np.linalg.norm(diff_v[1][l]*C_root[1][l])**2/4      
        return min(1, np.exp(accept_prop)),ll_v,diff_v
#     elif method=='CNL':
#         diff_v = diff_ll(data)
#         for l in range(n_layer+1):
#             accept_prop += -np.sum((xi_v[0][l]-xi_u[0][l])*diff_u[0][l])/2 - delta*np.sum((xi_u[0][l]+xi_v[0][l])*diff_u[0][l]/C[0][l])/4 + delta*np.linalg.norm(diff_u[0][l])**2/4
#             accept_prop += -np.sum((xi_v[1][l]-xi_u[1][l])*diff_u[1][l])/2 - delta*np.sum((xi_u[1][l]+xi_v[1][l])*diff_u[1][l]/C[1][l])/4 + delta*np.linalg.norm(diff_u[1][l])**2/4

#             accept_prop -= -np.sum((xi_u[0][l]-xi_v[0][l])*diff_v[0][l])/2 - delta*np.sum((xi_v[0][l]+xi_u[0][l])*diff_v[0][l]/C[0][l])/4 + delta*np.linalg.norm(diff_v[0][l])**2/4  
#             accept_prop -= -np.sum((xi_u[1][l]-xi_v[1][l])*diff_v[1][l])/2 - delta*np.sum((xi_v[1][l]+xi_u[1][l])*diff_v[1][l]/C[1][l])/4 + delta*np.linalg.norm(diff_v[1][l])**2/4      
#         return min(1, np.exp(accept_prop)),ll_v,diff_v
    else:
        return min(1, np.exp(accept_prop)),ll_v   
    
def propose(xi,diff=False):
    w = xi[0]
    b = xi[1]
    
    noise = sample_prior()
    w_noise = noise[0]
    b_noise = noise[1]
    
    w_proposal = []
    b_proposal = []
    for l in range(n_layer+1):
        w_proposal.append(np.zeros(model[2*l].weight.shape))
        b_proposal.append(np.zeros(model[2*l].bias.shape))
    
    if method=='pCNL':
        diff_w = diff[0]
        diff_b = diff[1]
        for l in range(n_layer+1):
            w_proposal[l] = ((2-delta)*w[l] + 2*delta*C[0][l]*diff_w[l] + np.sqrt(8*delta)*w_noise[l])/(2+delta)
            b_proposal[l] = ((2-delta)*b[l] + 2*delta*C[1][l]*diff_b[l] + np.sqrt(8*delta)*b_noise[l])/(2+delta)
#     elif method=='CNL':
#         diff_w = diff[0]
#         diff_b = diff[1]
#         for l in range(n_layer+1):
#             w_proposal[l] = ((2*C[0][l]-delta)*w[l] + 2*delta*C[0][l]*diff_w[l] + np.sqrt(8*delta*C[0][l])*w_noise[l])/(2*C[0][l]+delta)
#             b_proposal[l] = ((2*C[1][l]-delta)*b[l] + 2*delta*C[1][l]*diff_b[l] + np.sqrt(8*delta*C[1][l])*b_noise[l])/(2*C[1][l]+delta)
    else:
        for l in range(n_layer+1):
            w_proposal[l] = np.sqrt(1-beta*beta)*w[l]+beta*w_noise[l]
            b_proposal[l] = np.sqrt(1-beta*beta)*b[l]+beta*b_noise[l]
    return [w_proposal,b_proposal]

def MCMC(xi,N_data,data,max_time):   
    print('\nMCMC algorithm ('+method + ', N_data=' + str(N_data) + ', ' + str(max_time) + ' seconds) was started: ' + str(time.ctime()))
      
    acc_ratio = 0
    logposterior = []
    logp = []
    logl = []
    
    ''' Set model weights and biases to current iterate '''
    if method=='pCNL' or method=='CNL':
        for l in range(n_layer+1):
            model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi[0][l]).float(), requires_grad=True)
            model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi[1][l]).float(), requires_grad=True)
    else:
        for l in range(n_layer+1):
            model[2*l].weight = torch.nn.Parameter(torch.from_numpy(xi[0][l]).float(), requires_grad=False)
            model[2*l].bias = torch.nn.Parameter(torch.from_numpy(xi[1][l]).float(), requires_grad=False)
        
    ''' Initialise likelihood and gradient ''' 
    ll = loglikelihood(data)
    print('Initial loglikelihood: ',ll)
    if method=='CNL' or method=='pCNL':
        diff = diff_ll(data)
    
    ''' Run MCMC '''
    start = time.time() 
    j = 0
    it = 0
    while(time.time()-start<max_time):
        
        ''' Swap functions around to get mode switching '''
        for _ in range(hidden_sizes[0]):
            xi = prior_proposal(xi)
            
        ''' Propose and calculate acceptance probability '''
        if method=='CNL' or method=='pCNL':
            xi_proposal = propose(xi,diff)  
            a,ll_proposal,diff_proposal = acceptance_prop(xi,xi_proposal,data,ll,diff)
        else:
            xi_proposal = propose(xi)  
            a,ll_proposal = acceptance_prop(xi,xi_proposal,data,ll)
        
        ''' Accept or reject proposal '''
        uni = np.random.uniform()
        if uni < a or unadjusted:
            if method=='CNL' or method=='pCNL':
                diff = diff_proposal
            xi = xi_proposal    
            ll = ll_proposal
            acc_ratio = acc_ratio + 1

        ''' prior, likelihood, and posterior traceplots are appended '''
        lp = logprior(xi)
        logposterior.append(lp+ll)
        logp.append(lp)
        logl.append(ll)
        
        if prior_compare and j%10==0:
            ''' store value function evaluations for uncertainty estimates '''
            test_value_fn()
        elif policy_compare and (time.time()-start)>it*t_max/1000 and it<1000:
            ''' store sample for future use '''
            for l in range(n_layer+1):
                np.save('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_w'+str(l)+'_sampleNo'+str(it)+'.npy',xi[0][l])
                np.save('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_b'+str(l)+'_sampleNo'+str(it)+'.npy',xi[1][l])
            it += 1
        
        if method=='pCNL' or method=='CNL':
            progressBar(time.time()-start,j+1,max_time,acc_ratio)
        elif (j+1)%100==0:
            progressBar(time.time()-start,j+1,max_time,acc_ratio)
        j+=1
        
    progressBar(max_time,j,max_time,acc_ratio)
    
    acc_ratio = acc_ratio/(j)
    print('\nMCMC algorithm terminated: ' + str(time.ctime()) + '. \nRuntime = ' + str(time.time()-start))
    print('Final loglikelihood: ',ll)
    print('Acceptance ratio is ',acc_ratio)

    for l in range(n_layer+1):
        np.save('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_w'+str(l)+'.npy',xi[0][l])
        np.save('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_b'+str(l)+'.npy',xi[1][l])
        
    trajectory_plot(logposterior[1:],'figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_logposterior')
    trajectory_plot(logp[1:],'figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_logprior')
    trajectory_plot(logl[1:],'figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_loglikelihood')
    if prior_compare:
        boxplot_value_fn()
        
    func_plot(xi,'figs/MC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample')
        
    ESS(logposterior,'figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_autocorr')

# MAIN PROGRAMMES

In [10]:
"""

MAIN PROGRAMME 1 - compare NN prior to KL prior (large number of parameters)

"""    

prior_compare = True
policy_compare = False  

# set maximal runtime
t_max = 3600*24

optimal_choice = 0
not_optimal_choice = 0
v_all = np.zeros((N_a,100))

''' Initialise network network, see second block for detailed comments '''
input_size = dim
n_layer = 3
hidden_sizes = [100,100,100]
output_size = 1
hyps = str(hidden_sizes[0])
for i in range(1,n_layer):
    hyps = hyps+'_'+str(hidden_sizes[i])
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[-1], output_size))
weights = []
biases = []
for l in range(n_layer+1):
    weights.append(np.zeros(model[2*l].weight.shape))
    biases.append(np.zeros(model[2*l].bias.shape))
for l in range(n_layer+1):
    model[2*l].weight = torch.nn.Parameter(torch.from_numpy(0*weights[l]).float(), requires_grad=False)
    model[2*l].bias = torch.nn.Parameter(torch.from_numpy(0*biases[l]).float(), requires_grad=False)
''' Initialise covariance operator, see prior block for detailed comments '''
C = [[],[]]
C_root = [[],[]]
C_arr = np.ones(model[0].weight.shape)
for t in range(hidden_sizes[0]):
    for s in range(input_size):
        C_arr[t][s] = sigma_w_sq[0]/np.power(t+1,alpha)
C[0].append(C_arr)
C_root[0].append(C_arr**(1/2))
for l in range(1,n_layer):
    C_arr = np.ones(model[2*l].weight.shape)
    for t in range(hidden_sizes[l]):
        for s in range(hidden_sizes[l-1]):
            C_arr[t][s] = sigma_w_sq[l]/np.power(s+1,alpha)/np.power(t+1,alpha)
    C[0].append(C_arr)
    C_root[0].append(C_arr**(1/2))
C_arr = np.ones(model[2*n_layer].weight.shape)   
for t in range(output_size):
    for s in range(hidden_sizes[n_layer-1]):
        C_arr[t][s] = sigma_w_sq[n_layer]/np.power(s+1,alpha)
C[0].append(C_arr)   
C_root[0].append(C_arr**(1/2))
C_arr = np.ones(model[0].bias.shape)
for t in range(hidden_sizes[0]):
    C_arr[t] = sigma_b_sq[0]/np.power(t+1,alpha)
C[1].append(C_arr)
C_root[1].append(C_arr**(1/2))
for l in range(1,n_layer):
    C_arr = np.ones(model[2*l].bias.shape)
    for t in range(hidden_sizes[l]):
        C_arr[t] = sigma_b_sq[l]/np.power(t+1,alpha)
    C[1].append(C_arr)
    C_root[1].append(C_arr**(1/2))
C_arr = np.ones(model[2*n_layer].bias.shape)
for t in range(output_size):
    C_arr[t] = sigma_b_sq[n_layer]/np.power(t+1,alpha)
C[1].append(C_arr)
C_root[1].append(C_arr**(1/2))

env = gym.make('HalfCheetah-v2')
env.unwrapped
env.reset()
    
try:
    data = np.load('HC_data.npy')
except FileNotFoundError:
    print('\nNo data found - create data!\n')
N_data = 100

''' run pCN '''
method = 'pCN'
stochastic_gradients = False
unadjusted = False
beta = 1/59
counts = np.zeros(2)
try:
    xi = [[],[]]
    for l in range(n_layer+1):
        xi[0].append(np.load('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_w'+str(l)+'.npy'))
        xi[1].append(np.load('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_b'+str(l)+'.npy')) 
except FileNotFoundError:
    print('\nStarting from close to 0')
    xi = sample_prior(beta)
func_plot(xi,'figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_firstSample')
MCMC(xi,N_data,data[50:50+N_data,:],t_max) 
print('Counts: '+str(int(counts[0]))+' accepted prior moves, '+str(int(counts[1]))+' rejected ones.')

env.close()


MCMC algorithm (pCN, N_data=100, 86400 seconds) was started: Tue Jul 14 23:08:48 2020
Initial loglikelihood:  -45.06797553436477
Iteration: 225103    Acceptance ratio: 0.294    Percent: [--------------------------------------->] 100%  
MCMC algorithm terminated: Wed Jul 15 23:08:48 2020. 
Runtime = 86400.06720089912
Final loglikelihood:  -54.246173549495246
Acceptance ratio is  0.2938210508078524
Samples, Correct choices:  0.3210226111678735
Samples, Wrong choices:    0.6789773888321265
Mean,    Correct choices:  0.42
Mean,    Wrong choices:    0.58

Effective Sample Size: 10.0
Samples required to generate 1 independent sample: 21769.06
Counts: 294653 accepted prior moves, 22215647 rejected ones.


In [11]:
"""

MAIN PROGRAMME 2 - LEARN policy, and store samples for future use (small number of parameters)

"""    

prior_compare = False
policy_compare = True   

# set maximal runtime
t_max = 3600*24

''' Initialise network network, see second block for detailed comments '''
input_size = dim
n_layer = 3
hidden_sizes = [10,10,10]
output_size = 1
hyps = str(hidden_sizes[0])
for i in range(1,n_layer):
    hyps = hyps+'_'+str(hidden_sizes[i])
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[-1], output_size))
weights = []
biases = []
for l in range(n_layer+1):
    weights.append(np.zeros(model[2*l].weight.shape))
    biases.append(np.zeros(model[2*l].bias.shape))
for l in range(n_layer+1):
    model[2*l].weight = torch.nn.Parameter(torch.from_numpy(0*weights[l]).float(), requires_grad=False)
    model[2*l].bias = torch.nn.Parameter(torch.from_numpy(0*biases[l]).float(), requires_grad=False)
''' Initialise covariance operator, see prior block for detailed comments '''
C = [[],[]]
C_root = [[],[]]
C_arr = np.ones(model[0].weight.shape)
for t in range(hidden_sizes[0]):
    for s in range(input_size):
        C_arr[t][s] = sigma_w_sq[0]/np.power(t+1,alpha)
C[0].append(C_arr)
C_root[0].append(C_arr**(1/2))
for l in range(1,n_layer):
    C_arr = np.ones(model[2*l].weight.shape)
    for t in range(hidden_sizes[l]):
        for s in range(hidden_sizes[l-1]):
            C_arr[t][s] = sigma_w_sq[l]/np.power(s+1,alpha)/np.power(t+1,alpha)
    C[0].append(C_arr)
    C_root[0].append(C_arr**(1/2))
C_arr = np.ones(model[2*n_layer].weight.shape)   
for t in range(output_size):
    for s in range(hidden_sizes[n_layer-1]):
        C_arr[t][s] = sigma_w_sq[n_layer]/np.power(s+1,alpha)
C[0].append(C_arr)   
C_root[0].append(C_arr**(1/2))
C_arr = np.ones(model[0].bias.shape)
for t in range(hidden_sizes[0]):
    C_arr[t] = sigma_b_sq[0]/np.power(t+1,alpha)
C[1].append(C_arr)
C_root[1].append(C_arr**(1/2))
for l in range(1,n_layer):
    C_arr = np.ones(model[2*l].bias.shape)
    for t in range(hidden_sizes[l]):
        C_arr[t] = sigma_b_sq[l]/np.power(t+1,alpha)
    C[1].append(C_arr)
    C_root[1].append(C_arr**(1/2))
C_arr = np.ones(model[2*n_layer].bias.shape)
for t in range(output_size):
    C_arr[t] = sigma_b_sq[n_layer]/np.power(t+1,alpha)
C[1].append(C_arr)
C_root[1].append(C_arr**(1/2))

env = gym.make('HalfCheetah-v2')
env.unwrapped
env.reset()

try:
    data = np.load('HC_data.npy')
except FileNotFoundError:
    print('\nNo data found - create data!\n')
N_data = 100

    
''' run pCN '''
method = 'pCN'
stochastic_gradients = False
unadjusted = False
beta = 1/59 #1/60 gave 25.2 % acc
counts = np.zeros(2)
try:
    xi = [[],[]]
    for l in range(n_layer+1):
        xi[0].append(np.load('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_w'+str(l)+'.npy'))
        xi[1].append(np.load('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_b'+str(l)+'.npy')) 
except FileNotFoundError:
    print('\nStarting from close to 0')
    xi = sample_prior(beta)
func_plot(xi,'figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_firstSample')
MCMC(xi,N_data,data[50:50+N_data,:],t_max) 
print('Counts: '+str(int(counts[0]))+' accepted prior moves, '+str(int(counts[1]))+' rejected ones.')
    
    
''' run pCNL ''' # can be modified to CNL by just replacing the method to 'CNL'
method = 'pCNL'
stochastic_gradients = False # if true then unadjusted needs to be true too
unadjusted = False
delta = 1/36000
counts = np.zeros(2)
try:
    xi = [[],[]]
    for l in range(n_layer+1):
        xi[0].append(np.load('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_w'+str(l)+'.npy'))
        xi[1].append(np.load('np_saved/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_lastSample_b'+str(l)+'.npy'))
except FileNotFoundError:
    print('\nStarting from close to 0')
    xi = sample_prior(delta)
func_plot(xi,'figs/HC/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_firstSample')
MCMC(xi,N_data,data[50:50+N_data,:],t_max) 
print('Counts: '+str(int(counts[0]))+' accepted prior moves, '+str(int(counts[1]))+' rejected ones.')


env.close()


MCMC algorithm (pCN, N_data=100, 86400 seconds) was started: Wed Jul 15 23:09:04 2020
Initial loglikelihood:  -33.97670185393364
Iteration: 245349    Acceptance ratio: 0.214    Percent: [--------------------------------------->] 100%  
MCMC algorithm terminated: Thu Jul 16 23:09:04 2020. 
Runtime = 86400.02739691734
Final loglikelihood:  -41.862402080688135
Acceptance ratio is  0.21368744115525232

Effective Sample Size: 22.0
Samples required to generate 1 independent sample: 11107.07
Counts: 605615 accepted prior moves, 1847875 rejected ones.

MCMC algorithm (pCNL, N_data=100, 86400 seconds) was started: Thu Jul 16 23:09:20 2020
Initial loglikelihood:  -51.575600068520444
Iteration: 26384    Acceptance ratio: 0.581    Percent: [--------------------------------------->] 100%  
MCMC algorithm terminated: Fri Jul 17 23:09:25 2020. 
Runtime = 86401.34112596512
Final loglikelihood:  -40.47636956370256
Acceptance ratio is  0.5810339599757429

Effective Sample Size: 7.0
Samples required to 